## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/firstdf/tips.csv"
file_type = "csv"

# CSV options
df=spark.read.csv( "/FileStore/tables/firstdf/tips.csv",header=True,inferSchema=True)

In [0]:
df.printSchema

Out[2]: <bound method DataFrame.printSchema of DataFrame[Total_bill: double, Tip: double, Sex: string, Smoker: string, Day: string, Time: string, Size: int]>

In [0]:
# Feature engineering

from pyspark.ml.feature import StringIndexer
df1.show(20)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4218491722824155> in <module>
      2 
      3 from pyspark.ml.feature import StringIndexer
----> 4 df1.show(20)

NameError: name 'df1' is not defined

In [0]:
indexer=StringIndexer(inputCols=['Smoker','Day','Time'],outputCols=['Smoker_indexed','Day_indexed','Time_indexed'])
df1=indexer.fit(df).transform(df)

In [0]:
df1.columns

In [0]:
from pyspark.ml.feature import VectorAssembler
features=VectorAssembler(inputCols=[ 'Tip',
 'Size',
 'Smoker_indexed',
 'Day_indexed',
 'Time_indexed'],outputCol='Indep Features')
model=features.transform(df1)

In [0]:
model.show()

In [0]:
final_model= model.select('Indep Features','Total_bill')
final_model.show()

In [0]:
from pyspark.ml.regression import LinearRegression
# train test
train,test= final_model.randomSplit([0.75,0.25])
reg= LinearRegression(featuresCol='Indep Features',labelCol='Total_bill')
reg=reg.fit(train)

In [0]:
reg.coefficients

In [0]:
reg.intercept

In [0]:
# prediction
pred=reg.evaluate(test)

In [0]:
pred.predictions.show()

In [0]:
pred.r2,pred.meanAbsoluteError,pred.meanSquaredError

In [0]:
pickleRdd = sc.pickleFile(reg).collect()
df2 = spark.createDataFrame(pickleRdd)